## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(input_shape=(32, 32, 3), kernel_size=(3,3), filters=32, activation='relu', padding='same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(kernel_size=(3,3), filters=32, activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))


#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 297s 6ms/step - loss: 1.3946 - accuracy: 0.5104
Epoch 2/100
50000/50000 [==============================] - 263s 5ms/step - loss: 0.9493 - accuracy: 0.6627
Epoch 3/100
50000/50000 [==============================] - 262s 5ms/step - loss: 0.7764 - accuracy: 0.7257
Epoch 4/100
50000/50000 [==============================] - 255s 5ms/step - loss: 0.6483 - accuracy: 0.7699
Epoch 5/100
50000/50000 [==============================] - 255s 5ms/step - loss: 0.5213 - accuracy: 0.8155
Epoch 6/100
50000/50000 [==============================] - 252s 5ms/step - loss: 0.4195 - accuracy: 0.8549
Epoch 7/100
50000/50000 [==============================] - 272s 5ms/step - loss: 0.3204 - accuracy: 0.8881
Epoch 8/100
50000/50000 [==============================] - 272s 5ms/step - loss: 0.2351 - accuracy: 0.9189
Epoch 9/100
50000/50000 [==============================] - 263s 5ms/step - loss: 0.1803 - accuracy: 0.9369
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 296s 6ms/step - loss: 0.0184 - accuracy: 0.9945
Epoch 77/100
50000/50000 [==============================] - 283s 6ms/step - loss: 0.0140 - accuracy: 0.9958
Epoch 78/100
50000/50000 [==============================] - 268s 5ms/step - loss: 0.0173 - accuracy: 0.9944
Epoch 79/100
50000/50000 [==============================] - 244s 5ms/step - loss: 0.0170 - accuracy: 0.9944
Epoch 80/100
50000/50000 [==============================] - 244s 5ms/step - loss: 0.0151 - accuracy: 0.9953
Epoch 81/100
50000/50000 [==============================] - 278s 6ms/step - loss: 0.0175 - accuracy: 0.9945
Epoch 82/100
50000/50000 [==============================] - 381s 8ms/step - loss: 0.0176 - accuracy: 0.9946
Epoch 83/100
50000/50000 [==============================] - 365s 7ms/step - loss: 0.0185 - accuracy: 0.9943
Epoch 84/100
50000/50000 [==============================] - 305s 6ms/step - loss: 0.0098 - accuracy: 0.9965
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.6770250e-09, 1.3585649e-25, 6.9778965e-13, 4.2221704e-03,
        9.9577785e-01, 4.1717559e-29, 2.6708230e-23, 7.3105261e-21,
        9.8505094e-14, 2.2676692e-21]], dtype=float32)